In [4]:
import pandas as pd
import numpy as np

In [101]:
df1 = pd.read_csv('인프라\\서울특별시 광진구_공영주차장 정보_20220118.csv')

In [9]:
df2 = pd.read_csv('인프라\\서울특별시 광진구_가로등 위치정보_20220214.csv')

In [12]:
import folium
df = pd.read_csv('인프라_3개제외.csv')
df[['폐업일자','영업상태명']] = df[['폐업일자','영업상태명']].fillna(0)
test = df.dropna()
a = []
for i in range( test.shape[0]):
    arr =[test.iloc[i][1],test.iloc[i][2],test.iloc[i][-1]]
    a.append(arr)

In [14]:
test.reset_index(inplace=True,drop=True)

In [15]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
# Geopandas 지오데이터프레임 생성
gdf = gpd.GeoDataFrame(test, geometry=gpd.points_from_xy(test['좌표정보(X)'], test['좌표정보(Y)']))

# 격자 생성
xmin, ymin, xmax, ymax = [gdf.total_bounds[i] for i in range(4)]
width = height = 0.0089 # 대략 1km
rows = int((ymax-ymin) / height)
cols = int((xmax-xmin) / width)
grid = []
for i in range(cols):
    for j in range(rows):
        grid.append(Polygon([
            (xmin + i * width, ymin + j * height),
            (xmin + (i + 1) * width, ymin + j * height),
            (xmin + (i + 1) * width, ymin + (j + 1) * height),
            (xmin + i * width, ymin + (j + 1) * height)
        ]))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf, grid_gdf, how='left', op='within')

# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

# 결과 출력 혹은 저장

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [16]:
joined

,인허가일자,좌표정보(X),좌표정보(Y),폐업일자,영업상태명,정보,geometry,index_right
0,1992-07-22,127.091354,37.538181,19990330,폐업,서울시광진구골프연습장업인허가정보.csv,POINT (127.09135 37.53818),115665.0
1,1993-02-27,127.106577,37.547179,19980703,폐업,서울시광진구골프연습장업인허가정보.csv,POINT (127.10658 37.54718),116584.0
2,1993-05-18,127.091249,37.537586,20230130,폐업,서울시광진구골프연습장업인허가정보.csv,POINT (127.09125 37.53759),115665.0
3,1994-01-27,127.081989,37.555756,20080422,폐업,서울시광진구골프연습장업인허가정보.csv,POINT (127.08199 37.55576),115208.0
4,1995-01-14,127.095874,37.538627,0,영업/정상,서울시광진구골프연습장업인허가정보.csv,POINT (127.09587 37.53863),116124.0
...,...,...,...,...,...,...,...,...
27261,2023-10-25,127.066891,37.535869,0,영업/정상,서울시광진구휴게음식점인허가정보.csv,POINT (127.06689 37.53587),114746.0
27262,2023-10-25,127.061296,37.536253,0,영업/정상,서울시광진구휴게음식점인허가정보.csv,POINT (127.06130 37.53625),114287.0
27263,2023-10-27,127.085362,37.557365,0,영업/정상,서울시광진구휴게음식점인허가정보.csv,POINT (127.08536 37.55737),115667.0
27264,2023-11-02,127.078977,37.555599,0,영업/정상,서울시광진구휴게음식점인허가정보.csv,POINT (127.07898 37.55560),115208.0


In [102]:
insert = '서울시 광진구 '

In [103]:
df1['위치'] = insert + df1['위치']

In [104]:
df1.loc[df1['위치'].str.contains('일대'),'위치']

5       서울시 광진구 중곡4동 453 일대
20    서울시 광진구 구의3동 609-1 일대
28      서울시 광진구 자양3동 712 일대
Name: 위치, dtype: object

In [105]:
df1.loc[df1['위치'].str.contains('일대'),'위치'] = df1[df1['위치'].str.contains('일대')]['위치'].str.slice(stop=-2)

In [106]:
df1.loc[df1['위치'].str.contains('외'),'위치'] = df1[df1['위치'].str.contains('외')]['위치'].str.slice(stop=-2)

In [107]:
import geocoder

In [64]:
geo_dict = {}
for i  in df1['위치']:
    geo_value = geocoder.osm(i)
    geo_dict[i] = [geo_value.lat,geo_value.lng]
    geo_list = [geo_value.lat,geo_value.lng]

In [108]:
len(geo_dict)

36

In [131]:
for i in geo_dict.keys():
    df1.loc[df1['위치'] ==  i,['위도','경도']] = geo_dict[i]

In [134]:
gdf_2 = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['경도'], df1['위도']))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf_2, grid_gdf, how='left', op='within')
# 기존 격자 기준에 매핑
# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [136]:
joined.to_csv('격자화_공영주차장.csv',index=False)

---

In [137]:
df2

,관리번호,위도,경도,정보
0,로데오거리1-1,37.539513,127.070008,서울특별시 광진구_가로등 위치정보_20220214.csv
1,로데오거리1-16,37.540853,127.066288,서울특별시 광진구_가로등 위치정보_20220214.csv
2,중곡제방길1,37.571695,127.079518,서울특별시 광진구_가로등 위치정보_20220214.csv
3,중곡제방길10,37.570218,127.079153,서울특별시 광진구_가로등 위치정보_20220214.csv
4,중곡제방길11,37.570053,127.079086,서울특별시 광진구_가로등 위치정보_20220214.csv
...,...,...,...,...
2717,하우스스토리1-5,37.530094,127.089264,서울특별시 광진구_가로등 위치정보_20220214.csv
2718,하우스스토리1-6,37.529902,127.089319,서울특별시 광진구_가로등 위치정보_20220214.csv
2719,하우스스토리1-7,37.529831,127.089568,서울특별시 광진구_가로등 위치정보_20220214.csv
2720,하우스스토리1-8,37.529883,127.090268,서울특별시 광진구_가로등 위치정보_20220214.csv


In [140]:
gdf_3 = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2['경도'], df2['위도']))

# 격자별로 데이터 포인트 매핑
grid_gdf = gpd.GeoDataFrame(geometry=grid)
joined = gpd.sjoin(gdf_3, grid_gdf, how='left', op='within')
# 기존 격자 기준에 매핑
# 격자별 데이터 집계
aggregated_data = joined.groupby('index_right').size()

C:\Users\HP\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [142]:
joined.to_csv('격자화_가로등.csv',index=False)